In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
df = pd.read_csv("../Data/weather_data_raw.csv")

print("Size of dataset:", df.shape)
df.head()

Size of dataset: (14262, 7)


,STATION,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyVisibility,HourlyWindSpeed
0,72784524163,2025-04-01T00:53:00,35,41,79,10,10
1,72784524163,2025-04-01T01:53:00,34,41,76,10,0
2,72784524163,2025-04-01T02:53:00,33,39,79,10,6
3,72784524163,2025-04-01T03:53:00,33,40,77,10,10
4,72784524163,2025-04-01T04:53:00,33,39,79,10,6


In [9]:
df.info()
df.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14262 entries, 0 to 14261
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   STATION                    14262 non-null  int64 
 1   DATE                       14262 non-null  object
 2   HourlyDewPointTemperature  13759 non-null  object
 3   HourlyDryBulbTemperature   13760 non-null  object
 4   HourlyRelativeHumidity     13759 non-null  object
 5   HourlyVisibility           13781 non-null  object
 6   HourlyWindSpeed            13744 non-null  object
dtypes: int64(1), object(6)
memory usage: 780.1+ KB


,STATION,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyVisibility,HourlyWindSpeed
count,1.426200e+04,14262,13759,13760,13759,13781,13744
unique,NaN,4386,57,75,88,38,31
top,NaN,2025-04-30T23:59:00,43,52,93,10,0
freq,NaN,10,850,698,636,11484,1658
mean,7.279091e+10,NaN,NaN,NaN,NaN,NaN,NaN
std,5.015093e+06,NaN,NaN,NaN,NaN,NaN,NaN
min,7.278452e+10,NaN,NaN,NaN,NaN,NaN,NaN
25%,7.278502e+10,NaN,NaN,NaN,NaN,NaN,NaN
50%,7.279302e+10,NaN,NaN,NaN,NaN,NaN,NaN
75%,7.279372e+10,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Check for duplicates
num_duplicates = df.duplicated().sum()
print("Number of duplicates in dataset is:", num_duplicates)

duplicates = df[df.duplicated()]
duplicates.head()


Number of duplicates in dataset is: 14


,STATION,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyVisibility,HourlyWindSpeed
759,72784524163,2025-04-30T23:59:00,NaN,NaN,NaN,NaN,NaN
1561,72784524163,2025-05-31T23:59:00,NaN,NaN,NaN,NaN,NaN
2314,72784524163,2025-06-30T23:59:00,NaN,NaN,NaN,NaN,NaN
3441,72793024233,2025-04-30T23:59:00,NaN,NaN,NaN,NaN,NaN
5627,72793024233,2025-06-30T23:59:00,NaN,NaN,NaN,NaN,NaN


In [11]:
# --- Remove duplicates and save to new CSV ---
df = df.drop_duplicates()
df.to_csv("wea_da_01.csv", index=False)

In [12]:
# --- Check missing values ---
df = pd.read_csv("wea_da_01.csv")
df.isnull().sum()


STATION                        0
DATE                           0
HourlyDewPointTemperature    489
HourlyDryBulbTemperature     488
HourlyRelativeHumidity       489
HourlyVisibility             467
HourlyWindSpeed              504
dtype: int64

In [13]:
# --- Clean DATE column: keep only rows where minute == 53 ---
# Convert to datetime
df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")

# Keep only rows where minute == 53
df = df[df["DATE"].dt.minute == 53]

# Reset index after filtering
df = df.reset_index(drop=True)

# Save the cleaned dataset
df.to_csv("wea_da_01.csv", index=False)


In [14]:
# --- Extract new features from DATE ---

# Ensure DATE is in datetime format
df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")

# Create new column 'date' with day/month only
df["date"] = df["DATE"].dt.strftime("%m-%d")

# Create new column 'hour' with only the hour value (integer)
df["hour"] = df["DATE"].dt.hour

# Save updated dataset
df.to_csv("wea_da_01.csv", index=False)


In [15]:
# --- Map STATION codes to airport codes ---
station_map = {
    "72797624217": "BLI",
    "72784524163": "PSC",
    "72785024157": "GEG",
    "72793724222": "PAE",
    "72793024233": "SEA"
}

df["STATION"] = df["STATION"].astype(str).map(station_map).fillna(df["STATION"])

# Save updated dataset
df.to_csv("wea_da_01.csv", index=False)


In [16]:
# --- Drop DATE and reorder columns ---
df = df.drop(columns=["DATE"])

# Reorder columns
cols_order = [
    "STATION",
    "date",
    "hour",
    "HourlyDewPointTemperature",
    "HourlyDryBulbTemperature",
    "HourlyRelativeHumidity",
    "HourlyVisibility",
    "HourlyWindSpeed"
]

df = df[cols_order]

# Save updated dataset
df.to_csv("wea_da_01.csv", index=False)


In [17]:
# --- Logical checks  ---
#Check if any date has more than 24 hours (duplicate hour in the same date) ---

# Count unique hours per date
hour_count = df.groupby("date")["hour"].nunique()
invalid_dates = hour_count[hour_count > 24]
if not invalid_dates.empty:
    print("Dates with more than 24 unique hours (duplicates detected):")
    print(invalid_dates)
else:
    print("No date has more than 24 hours.")

# Find dates with more less 24 hours
invalid_dates = hour_count[hour_count < 24]
if not invalid_dates.empty:
    print("Dates with less than 24 unique hours (duplicates detected):")
    print(invalid_dates)
else:
    print("No date has less than 24 hours.")

# Check if each hour appears only once per date
duplicates_per_hour = df.groupby(["date", "hour"]).size()
invalid_combinations = duplicates_per_hour[(duplicates_per_hour > 5) | (duplicates_per_hour < 5)] # (because of data has 5 different station)
if not invalid_combinations.empty:
    print("Not validate:")
    print(invalid_combinations)
else:
    print("All dates have each hour exactly once.")

No date has more than 24 hours.
No date has less than 24 hours.
Not validate:
date   hour
04-04  11      4
04-26  8       4
dtype: int64


In [18]:
# --- Check missing values ---
df.isnull().sum()

STATION                       0
date                          0
hour                          0
HourlyDewPointTemperature    28
HourlyDryBulbTemperature     27
HourlyRelativeHumidity       28
HourlyVisibility              8
HourlyWindSpeed              37
dtype: int64

In [19]:
# # --- Identify rows with missing values ---
# wea_missing = df[df.isnull().any(axis=1)]
# print("Number of rows with missing values:", wea_missing.shape[0])

# # Save rows with missing values
# wea_missing.to_csv("wea_missing_value.csv", index=False)


In [20]:
# --- Drop rows with missing values from main dataset ---
df_clean = df.dropna()

# Save cleaned dataset (no missing values)
df_clean.to_csv("..\Data\weather_data_clean.csv", index=False)